In [1]:
import sys
import os
sys.path.insert(1, '../scripts/')
import xarray
import openpyxl
import contextily as cx 
import yaml
import numpy as np
import pandas
import pathlib
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
# load functions from my scripts file "ssm_utils"
from ssm_utils import get_nearest_node, reshape_fvcom, calc_fvcom_stat, extract_fvcom_level

In [2]:
case = "SOG_NB"
scope= "benthic"

In [1]:
with open('../etc/SSM_config.yaml', 'r') as file:
    ssm = yaml.safe_load(file)
# get shapefile path    
shp = ssm['paths']['shapefile']
# load shapefile into geopandas dataframe
gdf = gpd.read_file(shp)
gdf=gdf.rename(columns={'region_inf':'Regions'})

NameError: name 'yaml' is not defined

In [8]:
test = np.array([1,2,3])
test.shape

(3,)

In [4]:
np_operator='min'
model_var='DOXG'
processed_netcdf_dir = pathlib.Path(ssm['paths']['processed_output'])/model_var
# get list of run sub-directories in processed netcdf directory
dir_list = os.listdir(processed_netcdf_dir)

In [5]:
%%time
MinBottomDO_full={}
MinBottDO={}
for run_dir in dir_list:
    try: 
        run_file=processed_netcdf_dir/run_dir/'bottom'/f'daily_{np_operator}_{model_var}_bottom.nc'
        with xarray.open_dataset(run_file) as ds:
            print([*ds])
            MinBottomDO_full[run_dir]=ds[f'{model_var}_daily_{np_operator}_bottom']
            # Sub-sample basin footprint (from 16012 nodes to 7494)
            MinBottDO[run_dir]=MinBottomDO_full[run_dir][:,gdf['node_id']-1]
            print(MinBottDO[run_dir].shape)
    except FileNotFoundError:
        print(f'File Not Found: {run_file}')

['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
['DOXG_daily_min_bottom']
(361, 4144)
CPU times: user 50.1 ms, sys: 45.8 ms, total: 96 ms
Wall time: 566 ms


In [17]:
def calc_hypoxic_ts(MinBottDO, DO_thresh, shp, dir_list, scope):
    """ 
    MinBottDO [2D]: Min daily bottom DOXG
    DO_thresh [1D or int]: DO_std or integer value
    shp [path]: shapefile path
    dir_list [list]: List of directory names for model output
    scope [string]: "benthic" or "wc" (for water column)
    """
    # Initialize
    Hypoxic={} # Boolean where DOXG<threshold
    HypoxicDays={} # Number of days where Hypoxic = True
    TotalHypoxic={} # Sum of days across regions
    VolumeDays={} # Percent of volume within region where DO<threshold
    PercentVolumeDays={}
    # Define dimension sizes and load shapefile
    [ndays,nnodes]=MinBottDO[dir_list[0]].shape
    gdf = gpd.read_file(shp)
    gdf=gdf.rename(columns={'region_inf':'Regions'})
    regions = gdf[['node_id','Regions']].groupby('Regions').count().index.to_list()
    if DO_thresh=='DO_standard':
        DO_thresh=gdf['DO_std']
    # Create array of Dissolved Oxygen threshold values 
    if type(DO_thresh)==int:
        DO_thresh2D = np.ones((nnodes,ndays))*DO_thresh
    else:
        # create array of DO_threshold values 
        DO_thresh2D = np.ones((nnodes,ndays))*np.array(DO_thresh).reshape(nnodes,1)
        # (7494,361) x (7494,1) => element-wise multiplication 
    # Calculate volume for volume days
    if scope=='benthic':
        volume = np.asarray(gdf.volume*ssm['siglev_diff'][-1]/100)
    else: # water column
        volume = np.asarray(gdf.volume)
    # Determine Hypoxic days
    for run_type in dir_list:
        print(run_type)
        # Boolean where DOXG<threshold
        Hypoxic[run_type] = MinBottDO[run_type]<=DO_thresh2D.transpose() #361x7494 (nodes x time)
        # Volume days
        VolumeDays=np.zeros((ndays,nnodes))
        for ti in range(0,ndays):
             VolumeDays[ti,:]=volume*Hypoxic[run_type][ti,:]
    return Hypoxic,VolumeDays

In [18]:
Hypoxic,VolumeDays=calc_hypoxic_ts(MinBottDO, 'DO_standard', shp, dir_list, scope)

wqm_reference
1c_all_sog_riv_off
2b_sog_river_2times
wqm_baseline
2a_sog_river_0.5times
1d_small_sog_wwtp_off
1e_med_sog_wwtp_off
1b_all_sog_wwtp_off


In [20]:
Hypoxic['wqm_reference']

<xarray.DataArray 'DOXG_daily_min_bottom' (dim_0: 361, dim_1: 4144)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])
Dimensions without coordinates: dim_0, dim_1